<a href="https://colab.research.google.com/github/AgnesElza/subscription-retention-analytics/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Subscription Retention Analysis & Churn Prediction (KKBox)

## Step 1: Define the Product Problem (business framing)

### Introduction
Retention is one of the most critical product metrics for subscription-based businesses.
In this project, we analyze KKBox's music streaming dataset to:
- Identify drivers of customer churn
- Build a predictive churn model
- Simulate an experiment to test retention strategies
- Translate insights into actionable business recommendations

**Business Problem:**  
How can we improve user retention by identifying who is at risk of churning and testing personalized engagement strategies?

**Success Metrics:**
- Product metric: Increase 30-day retention by 5%  
- Data science metrics: AUC > 0.75 for churn model, statistically significant uplift (p < 0.05) in simulated A/B test  

---

### Project Workflow
1. **Data Understanding** – Explore users, transactions, and activity logs  
2. **Data Preparation** – Clean and engineer features for churn modeling  
3. **Exploratory Analysis** – Identify behavior patterns linked with churn  
4. **Churn Prediction Model** – Train and evaluate predictive models  
5. **Causal Insights** – Apply methods to separate correlation vs. causation  
6. **Experiment Simulation** – Design and simulate an A/B test on retention strategies  
7. **Conclusions & Business Recommendations**  

---

### Dataset
This project uses the **WSDM–KKBox’s Churn Dataset**:  
- `members.csv` → demographics  
- `transactions.csv` → subscription history  
- `user_logs.csv` → daily activity  
- `train.csv` → churn labels  

Source: [Kaggle – KKBox Churn Prediction Challenge](https://www.kaggle.com/competitions/kkbox-churn-prediction-challenge/data)

---

## Step 2: Data Understanding

### Data Download & Project Setup

In [ ]:
# Mounting Google Drive in Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Core data & visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling basics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

# Notebook display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)

In [ ]:
# Creating a Project Folder in Drive
import os

# Path inside your Google Drive
data_path = "/content/drive/MyDrive/data_science_projects/kkbox_project/data"
os.makedirs(data_path, exist_ok=True)


Uncomment the below cells if you want to download dataset from Kaggle to your drive using Kaggle API.

In [ ]:
# # Download KKBox Data with Kaggle API into Drive
# # Since this data is from a challenge, you will have to accept the rules before downloading the data

# !pip install -q kaggle

# # Add your Kaggle API key (make sure kaggle.json is already uploaded )
# !mkdir -p ~/.kaggle
# !cp /content/drive/MyDrive/data_science_projects/kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# # Confirm CLI sees your account
# !kaggle config view
# # Quick sanity: can we hit Kaggle at all?
# !kaggle competitions list -s kkbox | head -n 20

In [ ]:
# # Download dataset directly into Drive
# !kaggle competitions download -c kkbox-churn-prediction-challenge -p "$project_path"

In [ ]:
# # Check the download
# os.listdir("/content/drive/MyDrive/data_science_projects/kkbox_project/data")

In [ ]:
# # Unzip into the same Drive folder
# !unzip -q "$project_path/kkbox-churn-prediction-challenge" -d "$project_path"

Uncomment after placing your data in your drive.

In [ ]:
# # Install 7zip tool
# !apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# import glob, subprocess, os, shlex

# data_path = "/content/drive/MyDrive/data_science_projects/kkbox_project/data"

# for z in glob.glob(os.path.join(data_path, "*.7z")):
#     cmd = f'7z x {shlex.quote(z)} -o{shlex.quote(data_path)} -y'
#     print("Extracting:", os.path.basename(z))
#     subprocess.run(cmd, shell=True, check=True)

# # Verify extraction
# [f for f in os.listdir(data_path) if f.endswith(".csv")]

Extracting: user_logs.csv.7z
Extracting: transactions.csv.7z
Extracting: train.csv.7z
Extracting: members_v3.csv.7z


['user_logs.csv', 'transactions.csv', 'train.csv', 'members_v3.csv']

### Initial Exploration & Sanity Checks

In [ ]:
# Load manageable datasets fully
data_path = "/content/drive/MyDrive/data_science_projects/kkbox_project/data"
train = pd.read_csv(f"{data_path}/train.csv")
members = pd.read_csv(f"{data_path}/members_v3.csv")
transactions = pd.read_csv(f"{data_path}/transactions.csv")

Since user_logs.csv is huge (28 GB), let’s start EDA with the manageable tables:

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992931 entries, 0 to 992930
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   msno      992931 non-null  object
 1   is_churn  992931 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.2+ MB


In [ ]:
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6769473 entries, 0 to 6769472
Data columns (total 6 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   msno                    object
 1   city                    int64 
 2   bd                      int64 
 3   gender                  object
 4   registered_via          int64 
 5   registration_init_time  int64 
dtypes: int64(4), object(2)
memory usage: 309.9+ MB


In [ ]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21547746 entries, 0 to 21547745
Data columns (total 9 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   msno                    object
 1   payment_method_id       int64 
 2   payment_plan_days       int64 
 3   plan_list_price         int64 
 4   actual_amount_paid      int64 
 5   is_auto_renew           int64 
 6   transaction_date        int64 
 7   membership_expire_date  int64 
 8   is_cancel               int64 
dtypes: int64(8), object(1)
memory usage: 1.4+ GB


📊 First Observations

* The churn label file (train.csv) covers ~1M users, which is a subset of the full 6.8M members.

* Transactions are much larger, meaning multiple rows per user (renewals, different plans).

* We’ll eventually need to merge on msno to combine user labels + demographics + subscription history.

#### Questions to Answer
1. What is the churn rate (`is_churn`) in the dataset?  
2. What information do we have about users (from `members`)?  
3. What patterns exist in transactions (monthly vs. yearly plans, cancellations)?  
4. How do these datasets link together (`msno` = user ID)?  

In [ ]:
# Check churn distribution (how imbalanced is it?)
train['is_churn'].value_counts(normalize=True)

,proportion
is_churn,
0,0.936077
1,0.063923


Churn distribution tells us

*   Churn rate: ~6.4%
*   Retention: ~93.6%

📊 What this means

* The dataset is highly imbalanced → most users did not churn.

* This is realistic (subscription services usually have lower churn than retention).

For modeling:

* Accuracy will be misleading (a dumb model predicting “no churn” gets 93%!).

* Focus on AUC, precision/recall, F1, PR-AUC.

* Class balancing strategies: oversampling (SMOTE), undersampling, or class weights.

In [ ]:
members.describe(include='all')

,msno,city,bd,gender,registered_via,registration_init_time
count,6769473,6.769473e+06,6.769473e+06,2339968,6.769473e+06,6.769473e+06
unique,6769473,NaN,NaN,2,NaN,NaN
top,isBt+JlgvZRNy6lxVr5vvuJ4lD00ofTaKyJ+uRnGcPg=,NaN,NaN,male,NaN,NaN
freq,1,NaN,NaN,1195355,NaN,NaN
mean,NaN,3.847358e+00,9.795794e+00,NaN,5.253069e+00,2.014518e+07
std,NaN,5.478359e+00,1.792590e+01,NaN,2.361398e+00,2.318601e+04
min,NaN,1.000000e+00,-7.168000e+03,NaN,-1.000000e+00,2.004033e+07
25%,NaN,1.000000e+00,0.000000e+00,NaN,4.000000e+00,2.014042e+07
50%,NaN,1.000000e+00,0.000000e+00,NaN,4.000000e+00,2.015101e+07
75%,NaN,4.000000e+00,2.100000e+01,NaN,7.000000e+00,2.016060e+07


In [ ]:
transactions.describe(include='all')

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
count,21547746,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07,2.154775e+07
unique,2363626,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,xm6fmAfgZx1OYUXaJuHOObD0H2EAtIktv9NYIVlaTf4=,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.893310e+01,3.133906e+01,1.398850e+02,1.419873e+02,8.519661e-01,2.015760e+07,2.015862e+07,3.976523e-02
std,NaN,3.507936e+00,3.035649e+01,1.309647e+02,1.324824e+02,3.551336e-01,6.155979e+03,7.755753e+03,1.954072e-01
min,NaN,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.015010e+07,1.970010e+07,0.000000e+00
25%,NaN,3.800000e+01,3.000000e+01,9.900000e+01,9.900000e+01,1.000000e+00,2.015091e+07,2.015102e+07,0.000000e+00
50%,NaN,4.100000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,2.016032e+07,2.016043e+07,0.000000e+00
75%,NaN,4.100000e+01,3.000000e+01,1.490000e+02,1.490000e+02,1.000000e+00,2.016092e+07,2.016103e+07,0.000000e+00


#### Data Quality Checks (Members & Transactions)

Before diving deeper into feature engineering, it’s important to check for:
- **Missing values** (gender is highly incomplete, bd has many invalid ages).  
- **Duplicates** (user profiles or repeated transactions).  
- **Coverage across tables** (do all churn-labeled users appear in members and transactions?).  
- **Sanity of ranges**:
  - Age (`bd`) has extreme outliers (clip to 10–100).  
  - Gender has many missing values (treat as “unknown”).  
  - Transaction dates align with membership expiration.  
  - Plan lengths and prices are within expected ranges.  

#### Members Exploration & Data Quality Checks

The members table provides user demographics and registration info.  
Key checks here:  
- Missing values and duplicates  
- Validity of age (`bd`)  
- Gender completeness  
- Registration channel distribution (`registered_via`)  
- Registration dates and coverage  

We’ll clean `bd` (clip to realistic range), fill missing genders as "unknown", and convert registration dates to datetime.

In [ ]:
# --- Missing values ---
print("Missing values per column:")
print(members.isnull().sum())

# --- Duplicates ---
print("\nNumber of duplicate rows:", members.duplicated().sum())

# --- Unique users ---
print("\nUnique users in members:", members['msno'].nunique())

# --- Gender distribution ---
print("\nGender distribution:")
print(members['gender'].value_counts(dropna=False))

# --- Age (bd) sanity check ---
print("\nAge stats (bd):")
print(members['bd'].describe())

# --- Registration channel ---
print("\nRegistered via distribution:")
print(members['registered_via'].value_counts())

# --- Convert registration date ---
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'], format='%Y%m%d')
print("\nRegistration date range:",
      members['registration_init_time'].min(), "→", members['registration_init_time'].max())


Missing values per column:
msno                            0
city                            0
bd                              0
gender                    4429505
registered_via                  0
registration_init_time          0
dtype: int64

Number of duplicate rows: 0

Unique users in members: 6769473

Gender distribution:
gender
NaN       4429505
male      1195355
female    1144613
Name: count, dtype: int64

Age stats (bd):
count    6.769473e+06
mean     9.795794e+00
std      1.792590e+01
min     -7.168000e+03
25%      0.000000e+00
50%      0.000000e+00
75%      2.100000e+01
max      2.016000e+03
Name: bd, dtype: float64

Registered via distribution:
registered_via
 4     2793213
 3     1643208
 9     1482863
 7      805895
 11      25047
 13       5455
 8        3982
 5        3115
 17       1494
 2        1452
 6        1213
 19        974
 16        888
 14        615
 1          43
 10         10
 18          5
-1           1
Name: count, dtype: int64

Registration date range:

### Members Cleaning

In [ ]:
# ---------- MEMBERS CLEAN ----------
members_clean = members.copy()

# Age: keep only 10–100, else NaN (numeric dtype preserved)
members_clean["bd"] = members_clean["bd"].apply(lambda x: x if 10 <= x <= 100 else np.nan)

# Gender: fill missing as 'unknown'
members_clean["gender"] = members_clean["gender"].fillna("unknown")

# Optional: guard against future/leaky dates (after dataset window end)
members_clean.loc[members_clean["registration_init_time"] > pd.Timestamp("2017-04-30"), "registration_init_time"] = pd.NaT

# Convenience features (cheap & useful)
members_clean["reg_year"]  = members_clean["registration_init_time"].dt.year
members_clean["reg_month"] = members_clean["registration_init_time"].dt.month

# Save
members_clean_path = f"{data_path}/members_clean.csv"
members_clean.to_csv(members_clean_path, index=False)
print("Saved:", members_clean_path, members_clean.shape)


Saved: /content/drive/MyDrive/data_science_projects/kkbox_project/data/members_clean.csv (6769473, 8)


📊Observations from members


Age (bd) is unreliable and gender is mostly missing, but registration channel and registration date are strong, usable features.
So the real signal in members will come from account age and signup channel, not demographics.



#### Transactions Exploration & Data Quality Checks

The transactions table tracks subscription payments and cancellations for each user.  
Key checks here:  
- Missing values and duplicates  
- Distribution of plan lengths and prices  
- Auto-renew vs. cancellations  
- Date ranges for transactions and membership expiry  

This will help us create per-user subscription features later (e.g., average plan length, churn signals from cancellations).

In [ ]:
# --- Missing values ---
print("Missing values per column:")
print(transactions.isnull().sum())

# --- Duplicates ---
print("\nNumber of duplicate rows:", transactions.duplicated().sum())

# --- Unique users covered ---
print("\nUnique users in transactions:", transactions['msno'].nunique())

# --- Distribution checks ---
print("\nPlan length (days):")
print(transactions['payment_plan_days'].value_counts().head(10))

print("\nAuto-renew flag:")
print(transactions['is_auto_renew'].value_counts())

print("\nCancellation flag:")
print(transactions['is_cancel'].value_counts())

print("\nPlan price stats:")
print(transactions['plan_list_price'].describe())
print("\nAmount actually paid stats:")
print(transactions['actual_amount_paid'].describe())

# --- Convert dates ---
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date'], format='%Y%m%d')
transactions['membership_expire_date'] = pd.to_datetime(transactions['membership_expire_date'], format='%Y%m%d')

print("\nTransaction date range:",
      transactions['transaction_date'].min(), "→", transactions['transaction_date'].max())

print("Membership expiry range:",
      transactions['membership_expire_date'].min(), "→", transactions['membership_expire_date'].max())


Missing values per column:
msno                      0
payment_method_id         0
payment_plan_days         0
plan_list_price           0
actual_amount_paid        0
is_auto_renew             0
transaction_date          0
membership_expire_date    0
is_cancel                 0
dtype: int64

Number of duplicate rows: 3339

Unique users in transactions: 2363626

Plan length (days):
payment_plan_days
30     18956290
0        870124
31       766608
7        577639
195      110234
410       80139
180       52272
10        38216
100       24154
90        12310
Name: count, dtype: int64

Auto-renew flag:
is_auto_renew
1    18357950
0     3189796
Name: count, dtype: int64

Cancellation flag:
is_cancel
0    20690895
1      856851
Name: count, dtype: int64

Plan price stats:
count    2.154775e+07
mean     1.398850e+02
std      1.309647e+02
min      0.000000e+00
25%      9.900000e+01
50%      1.490000e+02
75%      1.490000e+02
max      2.000000e+03
Name: plan_list_price, dtype: float64

Amount a

### Transactions Cleaning

📊Observations from transactions

- 30-day plans dominate; some invalid 0-day entries need cleaning.  
- ~85% of users auto-renew; non-renewals are potential churn signals.  
- ~4% transactions are cancellations → strong churn predictor.  
- Free/0-paid plans may signal trial churn.  
- Expiry dates have invalid 1970 (epoch start)entries → must be cleaned.  

In [ ]:
# ---------- TRANSACTIONS CLEAN ----------
tx_clean = transactions.copy()

# Drop exact duplicate rows (tiny, but harmless to remove)
tx_clean = tx_clean.drop_duplicates()

# Fix epoch/bad expiry dates (set to NaT if before year 2000)
tx_clean.loc[tx_clean["membership_expire_date"] < pd.Timestamp("2000-01-01"), "membership_expire_date"] = pd.NaT

# (Optional) clip absurd plan lengths; keep promos like 195/410
tx_clean.loc[tx_clean["payment_plan_days"] > 400, "payment_plan_days"] = 400

# Helper flags (useful later even if you already created in agg)
tx_clean["zero_paid"] = (tx_clean["actual_amount_paid"] == 0).astype(int)
tx_clean["is_30d"]    = (tx_clean["payment_plan_days"] == 30).astype(int)
tx_clean["discount_rate"] = np.where(
    tx_clean["plan_list_price"] > 0,
    1 - (tx_clean["actual_amount_paid"] / tx_clean["plan_list_price"]),
    np.nan
)


## Step 3: Feature Engineering

We want to turn raw logs/transactions/demographics into user-level features that help predict churn.

### Build per-user features from transactions

Goal: collapse many transaction rows per user into a single row with churn-relevant signals:
- activity: `n_txns`, `last_txn_date`
- plan behavior: `avg_plan_days`, `share_30d`, `auto_renew_rate`, `cancel_rate`
- payments: `avg_amount_paid`, `total_amount_paid`, `zero_paid_rate`, `avg_discount_rate`
- recency/coverage: `last_expire_date`, `last_gap_days = last_expire_date − last_txn_date`

We’ll also clean obvious bad dates (epoch 1970) and drop duplicates.

In [ ]:
# Helper columns
tx_clean['zero_paid'] = (tx_clean['actual_amount_paid'] == 0).astype(int)
tx_clean['is_30d'] = (tx_clean['payment_plan_days'] == 30).astype(int)
tx_clean['discount_rate'] = np.where(
    tx_clean['plan_list_price'] > 0,
    1 - (tx_clean['actual_amount_paid'] / tx_clean['plan_list_price']),
    np.nan
)

# --- Aggregate to one row per user ---
tx_agg = tx_clean.groupby('msno').agg(
    n_txns                = ('msno', 'size'),
    cancel_rate           = ('is_cancel', 'mean'),
    auto_renew_rate       = ('is_auto_renew', 'mean'),
    avg_plan_days         = ('payment_plan_days', 'mean'),
    std_plan_days         = ('payment_plan_days', 'std'),
    share_30d             = ('is_30d', 'mean'),
    zero_paid_rate        = ('zero_paid', 'mean'),
    avg_amount_paid       = ('actual_amount_paid', 'mean'),
    total_amount_paid     = ('actual_amount_paid', 'sum'),
    avg_list_price        = ('plan_list_price', 'mean'),
    avg_discount_rate     = ('discount_rate', 'mean'),
    last_txn_date         = ('transaction_date', 'max'),
    last_expire_date      = ('membership_expire_date', 'max'),
).reset_index()

# Post-agg feature: gap between last txn and expiry (days)
tx_agg['last_gap_days'] = (tx_agg['last_expire_date'] - tx_agg['last_txn_date']).dt.days

# Save for later merges
data_path = "/content/drive/MyDrive/data_science_projects/kkbox_project/data"
tx_agg_path = f"{data_path}/tx_agg.csv"
tx_agg.to_csv(tx_agg_path, index=False)

tx_agg.shape, tx_agg.head()

# Save
tx_agg_path = f"{data_path}/tx_agg.csv"
tx_agg.to_csv(tx_agg_path, index=False)
print("Saved:", tx_agg_path, members_clean.shape)


Saved: /content/drive/MyDrive/data_science_projects/kkbox_project/data/tx_agg.csv (6769473, 8)


What we have is a per-user aggregated transactions dataset — 236,3626 users (≈2.36M), each row describing their subscription/payment behavior.

- Each column is a new feature that didn’t exist before:

- Engagement with payments: n_txns, total_amount_paid, avg_plan_days

- Churn-related signals: cancel_rate, auto_renew_rate, zero_paid_rate

- Financial behavior: avg_amount_paid, avg_discount_rate

- Recency features: last_txn_date, last_expire_date, last_gap_days

Now we can merge these features into train (for the ~1M labeled churn users) to enrich our modeling dataset.

### Engagement Features from `user_logs.csv` (chunked)

Goal: turn daily logs (28GB) into per-user features without blowing RAM.

Features we’ll compute:
- `active_days` (nunique dates)
- `total_secs_sum`, `total_secs_mean`
- `total_listens` = sum of (num_25 + num_50 + num_75 + num_985 + num_100)
- `completion_ratio` = (num_985 + num_100) / total_listens
- `avg_unique_songs_per_day` = mean(num_unq)
- `max_unique_songs_per_day` = max(num_unq)
- `last_activity_date` (max date)

In [ ]:
from collections import defaultdict

data_path = "/content/drive/MyDrive/data_science_projects/kkbox_project/data"
logs_file = f"{data_path}/user_logs.csv"

# Read only needed cols; set dtypes to keep memory down
usecols = ["msno","date","num_25","num_50","num_75","num_985","num_100","num_unq","total_secs"]
dtypes = {
    "msno":"category",
    "date":"int32",
    "num_25":"int32","num_50":"int32","num_75":"int32","num_985":"int32","num_100":"int32",
    "num_unq":"int32",
    "total_secs":"float32",
}

# Accumulators
sum_total_secs   = defaultdict(float)
cnt_days         = defaultdict(int)     # we’ll compute nunique(date) with a helper set per chunk
sum_listens      = defaultdict(int)
sum_complete     = defaultdict(int)     # num_985 + num_100
sum_num_unq      = defaultdict(float)
max_num_unq      = defaultdict(int)
last_activity    = defaultdict(int)     # keep max yyyymmdd per user

# For active_days (nunique) at scale:
# We’ll compute per-chunk nunique dates and add them with a trick:
# count a user-date pair once per chunk, then sum across chunks using a set of seen keys per chunk only.
chunksize = 5_000_000  # drop to 2_000_000 if you hit memory errors

reader = pd.read_csv(logs_file, usecols=usecols, dtype=dtypes, chunksize=chunksize)
for i, chunk in enumerate(reader, start=1):
    # date -> datetime later; here keep as int for speed
    # Per-row helpers
    listens_cols = ["num_25","num_50","num_75","num_985","num_100"]
    chunk["total_listens"] = chunk[listens_cols].sum(axis=1)
    chunk["complete_count"] = chunk["num_985"] + chunk["num_100"]

    # Aggregate simple sums / means
    g = chunk.groupby("msno", observed=True)

    # Sums
    s_total_secs   = g["total_secs"].sum()
    s_listens      = g["total_listens"].sum()
    s_complete     = g["complete_count"].sum()
    s_sum_unq      = g["num_unq"].sum()
    s_max_unq      = g["num_unq"].max()
    s_last_date    = g["date"].max()

    # Nunique days per user in this chunk
    # (count unique dates per user within the chunk, then add to global)
    s_active_days  = g["date"].nunique()

    # Commit to dictionaries
    for k, v in s_total_secs.items():   sum_total_secs[k] += float(v)
    for k, v in s_listens.items():      sum_listens[k]    += int(v)
    for k, v in s_complete.items():     sum_complete[k]   += int(v)
    for k, v in s_sum_unq.items():      sum_num_unq[k]    += float(v)
    for k, v in s_max_unq.items():      max_num_unq[k]     = max(max_num_unq[k], int(v))
    for k, v in s_last_date.items():    last_activity[k]   = max(last_activity[k], int(v))
    for k, v in s_active_days.items():  cnt_days[k]       += int(v)

    if i % 5 == 0:
        print(f"Processed ~{i*chunksize:,} rows")

# Build dataframe
users = list(sum_total_secs.keys() | sum_listens.keys() | sum_complete.keys() | sum_num_unq.keys() | cnt_days.keys() | last_activity.keys())

logs_agg = pd.DataFrame({
    "msno": users,
    "total_secs_sum":   [sum_total_secs[u] for u in users],
    "total_listens":    [sum_listens[u]    for u in users],
    "complete_listens": [sum_complete[u]   for u in users],
    "sum_num_unq":      [sum_num_unq[u]    for u in users],
    "active_days":      [cnt_days[u]       for u in users],
    "max_unique_songs_per_day": [max_num_unq[u] for u in users],
    "last_activity_date_int":   [last_activity[u] for u in users],
})

# Derived features
logs_agg["total_secs_mean"] = logs_agg["total_secs_sum"] / logs_agg["active_days"].replace(0, np.nan)
logs_agg["avg_unique_songs_per_day"] = logs_agg["sum_num_unq"] / logs_agg["active_days"].replace(0, np.nan)
logs_agg["completion_ratio"] = logs_agg["complete_listens"] / logs_agg["total_listens"].replace(0, np.nan)

# Dates to datetime
logs_agg["last_activity_date"] = pd.to_datetime(logs_agg["last_activity_date_int"].astype("Int64").astype("string"),
                                                format="%Y%m%d", errors="coerce")
logs_agg = logs_agg.drop(columns=["sum_num_unq","last_activity_date_int"])

# Save
logs_agg_path = f"{data_path}/user_logs_agg.csv"
logs_agg.to_csv(logs_agg_path, index=False)

logs_agg.shape, logs_agg.head()

Processed ~25,000,000 rows
Processed ~50,000,000 rows
Processed ~75,000,000 rows
Processed ~100,000,000 rows
Processed ~125,000,000 rows
Processed ~150,000,000 rows
Processed ~175,000,000 rows
Processed ~200,000,000 rows
Processed ~225,000,000 rows
Processed ~250,000,000 rows
Processed ~275,000,000 rows
Processed ~300,000,000 rows
Processed ~325,000,000 rows
Processed ~350,000,000 rows
Processed ~375,000,000 rows


((5234111, 10),
                                            msno  total_secs_sum  \
 0  d8Hq2MBu9QFtrYs/QEUfJNzZFM4FlCEGMfPaYzreH0g=     2411.446114   
 1  yJ5Fj6RI9luDuya317Yk07ED7i+RCMx5eGm5vLW599E=     5879.036865   
 2  u9GYwkN8uYqDrfX5oxyUOGBU0RkH+0XJNxGF/77R45w=   138325.993286   
 3  7vs42vOwsT+aIlARQ2C9+21iKO+HBfXrcd78vjvrhqY=      205.000000   
 4  kUpDNFIDqtPys+nDYOlTO0IHAedOO7JEPULiiI5NW4U=      892.109024   
 
    total_listens  complete_listens  active_days  max_unique_songs_per_day  \
 0             17                 7            3                        11   
 1             45                24            2                        20   
 2            575               518           10                       119   
 3              1                 1            1                         1   
 4             10                 2            2                         7   
 
    total_secs_mean  avg_unique_songs_per_day  completion_ratio  \
 0       803.815371                  

In [ ]:
logs_agg_path = f"{data_path}/user_logs_agg.csv"
logs_agg.to_csv(logs_agg_path, index=False)
print("Saved:", logs_agg_path)

Saved: /content/drive/MyDrive/data_science_projects/kkbox_project/data/user_logs_agg.csv


We now have cleaned datasets (members_clean.csv, tx_agg.csv, user_logs_agg.csv).

In the next notebook (02_exploratory_data_analysis.ipynb), we’ll merge them with churn labels and analyze churn patterns.